In [1]:
import pandas as pd
import networkx as nx
import numpy as np
from collections import OrderedDict
from tqdm import tqdm

In [2]:
df = pd.read_csv('wiki-topcats-reduced.txt', sep='\t', header = None)
df.columns = ['from', 'to']

In [3]:
#vertexes
fr = df.groupby('from').count()
to = df.groupby('to').count()
V = set(list(fr.index) + list(to.index))
print('Number of nodes =', len(V))

Number of nodes = 461193


In [4]:
#directed cause these two are different
print(df[df['from'] ==52], '\n')
print(df[df['from'] ==401135].head())

   from       to
0    52   401135
1    52  1069112
2    52  1163551 

          from      to
361754  401135   60219
361755  401135  167532
361756  401135  400980
361757  401135  401018
361758  401135  401019


In [5]:
print('Number of edges:',len(df.index))

Number of edges: 2645247


In [6]:
print('The average degree of the vertex is:', len(df.index)/len(V))

The average degree of the vertex is: 5.735661642739591


In [7]:
o = open('wiki-topcats-categories.txt', 'r')
categories = {}
for line in o :
    line = line.replace('\n','')
    l = line.split(' ')
    l[0] = l[0].replace('Category:', '').replace(';','')
    if len(l[1:])>=3500:
        categories[l[0]] = set(list(map(int, l[1:]))).intersection(set(V))
            

In [10]:
input_category = list(categories.keys())[7]
input_articles = (set(categories[input_category]).intersection(set(V)))

In [11]:
DG = nx.from_pandas_edgelist(df, 'from', 'to', create_using=nx.DiGraph )

In [12]:
class Dijkstra:

    def dijkstra(self, dic, dist):
        node = dic
        actual_dist = 0
        try:
            l = [list(self.graph[node]),1]
            while len(l[0]) > 0:
                dist = l[1]
                for i in l[0]:
                    m = []
                    if (i in self.visited):
                        if self.actual_node in self.visited[i]:
                            if (self.visited[i][self.actual_node] > dist):
                                self.visited[i][self.actual_node] = dist
                        else:
                            self.visited[i][self.actual_node] = dist

                    elif (self.unvisited[i] == -1):
                        del self.unvisited[i]
                        self.visited[i] = {self.actual_node:dist}

                    for j in list(self.graph[i]):
                        if (not(j in self.visited)):
                            m.append(j)
                        elif not(self.actual_node in self.visited[j]):
                            m.append(j)
                l = [m,dist+1]
        except:
            print('damn')
    
    def __init__(self, graph, categories, input_category):
        
        self.graph = graph
        self.nodes = categories
        self.initial = input_category
        self.unvisited = {}
        for i in self.graph.nodes:
            self.unvisited[i] = -1
        self.visited = {}
        self.l = []
        idx = 1
        for i in self.initial:
            idx+=1
            self.actual_node = i
            self.visited[i] = {self.actual_node: 0}
            if i in self.unvisited:
                del self.unvisited[i]
            self.dijkstra(i,1)
            
        for i in self.initial:
                self.visited[i] = {i: 0}

In [13]:
dij = Dijkstra(DG, categories, input_articles)

In [14]:
median = {}
set_of_visited = set(list(dij.visited.keys()))
for i in categories:
    if i != input_category:
        shortest_path=[]
        s = set(categories[i]).intersection(set_of_visited)
        if len(s)> 0:
            for j in s:
                shortest_path += (list(dij.visited[j].values()))
            median[i] = np.median(shortest_path)
        else:
            median[i] = 100**100
median[input_category] = -1

In [15]:
block_ranking = OrderedDict()
block_ranking = OrderedDict(sorted(median.items(), key=lambda x: x[1]))
block_ranking

OrderedDict([('Year_of_birth_unknown', -1),
             ('Year_of_death_missing', 2.0),
             ('Year_of_birth_missing', 2.0),
             ('English_footballers', 3.0),
             ('Association_football_forwards', 3.0),
             ('Association_football_goalkeepers', 3.0),
             ('Association_football_midfielders', 3.0),
             ('Association_football_defenders', 3.0),
             ('Indian_films', 3.0),
             ('English_cricketers', 3.0),
             ('Rivers_of_Romania', 3.0),
             ('British_films', 3.0),
             ('Article_Feedback_Pilot', 4.0),
             ('The_Football_League_players', 5.0),
             ('Fellows_of_the_Royal_Society', 5.0),
             ('Members_of_the_United_Kingdom_Parliament_for_English_constituencies',
              6.0),
             ('Black-and-white_films', 6.0),
             ('Harvard_University_alumni', 7.0),
             ('People_from_New_York_City', 7.0),
             ('American_military_personnel_of_World

In [16]:
# map each article to the categories it is present

articles = {}
for name in categories:
    for article in categories[name]:
        if article in articles:
            articles[article].append(name)
        else:
            articles[article] = [name]

In [17]:
# if an article belongs to multiple categories, choose one according to block_ranking

for article in articles:
    if len(articles[article]) > 1:
        minimum = ''
        for cat in articles[article]:
            if (minimum == '') or (block_ranking[minimum] > block_ranking[cat]):
                minimum = cat
        articles[article] = [minimum]

In [18]:
categories_after_ranking = {}

for i in articles:
    if articles[i][0] in categories_after_ranking:
        categories_after_ranking[articles[i][0]] +=  [i]
    else:
        categories_after_ranking[articles[i][0]] =  [i]


In [19]:
# VERSIONE MODIFICATA
# da quella di prima cambia come trovare gli archi adatti (jj)

idx = 0
jj = nx.to_dict_of_lists(DG)

for name in block_ranking:
    if idx == 0:
        weight_dict = {}
        boh = DG.subgraph(categories_after_ranking[name])
        weight_dict[name] = {}
        for i in (boh.in_degree):
            for j in jj[i[0]]:
                    DG[i[0]][j]['weight'] = i[1]
            idx +=1
        for  i in boh.in_degree:
            weight_dict[name][i[0]] = i[1]
    else:
        try:
            boh = DG.subgraph(categories_after_ranking[name])
            weight_dict[name] = {}

            for i in boh.in_degree:
                cumsum = i[1]
                for j in jj[i[0]]:
                        try:
                            cumsum+=(list(DG.edges[j,i[0]].values()))[0]
                        except:
                            pass
                weight_dict[name][i[0]] = cumsum


            for i in boh.in_degree:
                for j in jj[i[0]]:
                        DG[i[0]][j]['weight'] = weight_dict[name][i[0]]
        except:
            pass

In [20]:
#rank per ogni nodo (come nello step3 dell'homework)
rank = []
for i in weight_dict:
    rank += list(OrderedDict(sorted(weight_dict[i].items(), key=lambda x: x[1], reverse=True)).keys())
